In [ ]:
!pip install torch_geometric
!pip install torchmetrics
!pip install rdkit
!pip install pytorch_lightning

In [2]:
import pandas as pd
import pytorch_lightning as pl
import torch
import torchmetrics
import torch.nn as nn

from torch_geometric.nn import GAT, VGAE
from torch_geometric.utils import from_smiles, negative_sampling, train_test_split_edges
from torch_geometric.transforms import RandomLinkSplit

from random import sample, shuffle
import numpy as np
import pickle

## Load the Data (Incomplete)
* requires conversion to proper format
* requires train-val-test split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load pubchem10M Dataset
PC10M_path = "/content/drive/Shareddrives/cs_89_network_science/data/pubchem10M/"
df_PC10M_train = pd.read_parquet(PC10M_path + "pubchem_10m_train.parquet")
df_PC10M_val = pd.read_parquet(PC10M_path + "pubchem_10m_val.parquet")

# Load clintox and tox21 smiles representations
df_clintox = pd.read_csv("/content/drive/Shareddrives/cs_89_network_science/data/clintox.csv")
df_tox21 = pd.read_csv("/content/drive/Shareddrives/cs_89_network_science/data/tox21.csv")

In [ ]:
# Convert SMILES to a graph data object with PyTorch Geometric
for i in range(10):
  print(from_smiles(df_clintox.get("smiles")[i]))

## Encoder

In [5]:
class GATEncoder(nn.Module):
  def __init__(self, in_channels, num_layers, hidden_dim=64):
    super().__init__()
    self.in_channels=in_channels
    self.embed_dim = hidden_dim
    self.num_layers = num_layers
    self.hidden_dim = hidden_dim

    self.layers = nn.ModuleList()
    self.gat = GAT(in_channels, hidden_channels=hidden_dim, num_layers=num_layers)

    self.mu = nn.Linear(hidden_dim, hidden_dim)
    self.logvar = nn.Linear(hidden_dim, hidden_dim)

  def forward(self, x_feats, edge_index, edge_attr):
    x_feats = self.gat(x=x_feats, edge_index=edge_index, edge_attr=edge_attr)
    #x_feats = self.mlp(x_feats)

    mu = self.mu(x_feats)
    logvar = self.logvar(x_feats)

    return mu, logvar

In [6]:
import pickle

## Define the VGAE
Current value for hidden channels is arbitrary. Layers are restricted to 2 due to general decrease in performance with subsequent layers.

In [8]:
from_smiles_xchannels = 9
input_features = from_smiles_xchannels

model = VGAE(encoder=GATEncoder(in_channels=input_features, hidden_dim=64, num_layers=2))

with open("/content/sample_data/16dim_pretrain_save_SMART", "rb") as f:
  model = pickle.load(f)['model']

#device = torch.device("cuda")  # necessary to utilize GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Set to run on", device, end=".")
model = model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 0
g_loss = -1

Set to run on cpu.

# Training


In [9]:
def train(cstart, cend, training_data, bigiter):
  model.train()
  optimizer.zero_grad()
  iter = bigiter[cstart:cend]
  print("[----------------------------------------------------------------------------------------------------]",
        end="\r[")
  pinterval = pmark = (len(iter))/101
  for idx in range(len(iter)):
    i = iter[idx]
    if idx > pmark:
      print("-", end='')
      pmark += pinterval
    data = from_smiles(training_data.get("smiles")[i])
    data = data.to(device)
    data.x = data.x.type(torch.float)
    # Likely preferable to aggregate in some manner. Leaving for now just to create structure of train/test functions
    # such that they run
    mu, logstd = model.forward(data.x, data.edge_index, edge_attr=data.edge_attr)
    model.__setattr__('__mu__', mu)
    model.__setattr__('__logstd__', logstd)
    loss = model.kl_loss()
    loss.backward()
  print("]", end ="\r")
  optimizer.step()

  global g_loss, epochs
  g_loss = loss
  epochs += 1

  return float(loss)

def test(cstart, cend, testing_data):
  model.eval()
  with torch.no_grad():
    iter = list(range(len(testing_data)))
    test_cases = iter[cstart:cend]
    divisor = len(iter)
    avgs = [0, 0]
    print("Test Eval:", end=" ")
    for i in test_cases:
      data = from_smiles(testing_data.get("smiles")[i])
      if data.edge_index.size()[1] > 0:
        data = data.to(device)
        data.x = data.x.type(torch.float)
        data.neg_edge_index = negative_sampling(data.edge_index)
        z = model.encode(data.x, data.edge_index, edge_attr=data.edge_attr)
        if z.size()[0] > 2:
          auc, ap = model.test(z, data.edge_index, data.neg_edge_index)
          if i < 3:
            print(round(auc, 2), round(ap, 2), end=", ")
          avgs[0] += auc
          avgs[1] += ap
        else:
          divisor -= 1
      else:
        divisor -= 1
    avgs = [x/divisor for x in avgs]
    print("AUC=" + str(round(avgs[0], 5)), "AP=" + str(round(avgs[1], 5)), end=".")
  print("")


In [ ]:
seg_size = 10000

for i in range(100):
  iter = list(range(len(df_PC10M_train)))
  shuffle(iter)
  j = 0
  while j < len(df_PC10M_train):
    print("Round", str(i+1) + "-" + str(int(j/seg_size+1)), "gives us:", train(j, j+seg_size, df_PC10M_train, iter))
    test(0, 1000, df_PC10M_val)
    j += seg_size
    torch.save({'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': g_loss,},
               "/content/drive/Shareddrives/cs_89_network_science/saves/Bmodel")
  print()

## Fine-tuning

In [ ]:
# Combine smiles representations from each
df_smiles = pd.concat([df[['smiles']] for df in [df_clintox, df_tox21]])
df_smiles = df_smiles.reset_index(drop=True)

# Split data for 70% training, 15% validation, 15% testing.
idx = list(range(len(df_smiles)))
shuffle(idx)
train_idx = idx[:int(0.7*len(df_smiles))]
val_idx = idx[int(0.7*len(df_smiles)):int(0.85*len(df_smiles))]
test_idx = idx[int(0.85*len(df_smiles)):]
df_smiles['train'] = df_smiles['test'] = df_smiles['val'] = 0
df_smiles.loc[train_idx, 'train'] = 1
df_smiles.loc[test_idx, 'test'] = 1
df_smiles.loc[val_idx, 'val'] = 1

#train_smiles = []
#for sr in df_smiles[df_smiles['train']==1].get("smiles"):
#  if sr not in train_smiles:
#    train_smiles.append(sr)
#
#test_smiles = []
#for sr in df_smiles[df_smiles['test']==1].get("smiles"):
#  if sr not in test_smiles:
#    test_smiles.append(sr)
#
#train_smiles = [from_smiles(sr) for sr in train_smiles]
#test_smiles = [from_smiles(sr) for sr in test_smiles]

df_smiles_train = df_smiles[df_smiles['train']==1]
df_smiles_train = df_smiles_train.reset_index(drop=True)
df_smiles_test = df_smiles[df_smiles['test']==1]
df_smiles_test = df_smiles_test.reset_index(drop=True)

In [ ]:
for i in range(100):
  iter = list(range(len(df_smiles_train)))
  shuffle(iter)
  print("Round", str(i+1), "gives us:", train(0, len(df_smiles_train), df_smiles_train, iter))
  test(0, len(df_smiles_test), df_smiles_test)

In [ ]:
torch.save({'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': g_loss,},
               "/content/drive/Shareddrives/cs_89_network_science/saves/Bmodel_finetuned")

## Classification

In [ ]:
def manual_sum_pooling(node_embeddings, batch):
    num_graphs = batch.max().item() + 1
    graph_embedding = torch.zeros((num_graphs, node_embeddings.size(1)), device=node_embeddings.device)
    for graph_id in range(num_graphs):
        graph_embedding[graph_id] = node_embeddings[batch == graph_id].sum(dim=0)
    return graph_embedding

# Define classifer model
class VGAEClassifier(nn.Module):
  def __init__(self, base_model, num_classes, lr=1e-5):
    super().__init__()
    self.base_model = base_model
    self.num_classes = num_classes
    self.fc = nn.Linear(base_model.encoder.hidden_dim, num_classes)
    self.lr = lr

    # Use BCEWithLogitsLoss for multi-label binary classification
    self.criterion = nn.BCEWithLogitsLoss()

    # Initialize metrics specifically for multi-label with the correct number of labels
    self.accuracy = torchmetrics.Accuracy(average='macro', task="multilabel", num_classes=num_classes, num_labels=num_classes)
    # For F1 and AUC, specifying num_classes for multilabel task
    self.f1_score = torchmetrics.F1Score(num_classes=num_classes, average='macro', task="multilabel", num_labels=num_classes)
    self.auc_roc = torchmetrics.AUROC(num_classes=num_classes, average='macro', task="multilabel", num_labels=num_classes)

  def forward(self, x, edge_index, edge_attr, batch=None):
    node_embeddings = self.base_model.encode(x, edge_index, edge_attr)

    if batch is not None:
        graph_embedding = manual_sum_pooling(node_embeddings, batch)
    else:
        graph_embedding = torch.sum(node_embeddings, dim=0)
    out = self.fc(graph_embedding)
    return out

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=self.lr)

In [ ]:
# Load the VGAE model
#model_vgae = VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=18, num_layers=2)) # 18 for Amodel; 64 for Bmodel q
model_vgae = VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=64, num_layers=2))

checkpoint = torch.load("/content/drive/Shareddrives/cs_89_network_science/saves/Bmodel_finetuned")
model_vgae.load_state_dict(checkpoint['model_state_dict'])

epochs = checkpoint['epoch']
g_loss = checkpoint['loss']

# Initialize the classifiers
model_clintox = VGAEClassifier(model_vgae, 2)
optimizer_clintox = model_clintox.configure_optimizers()

model_tox21 = VGAEClassifier(model_vgae, 12)
optimizer_tox21 = model_tox21.configure_optimizers()

# Set to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Set to run on", device, end=".")
model_clintox = model_clintox.to(device)
model_tox21 = model_tox21.to(device)

Set to run on cpu.

In [ ]:
# Split data for 70% training, 15% validation, 15% testing.
for df in [df_clintox, df_tox21]:
  n = len(df)
  idx = list(range(n))
  shuffle(idx)
  train_idx = idx[:int(0.7*n)]
  val_idx = idx[int(0.7*n):int(0.85*n)]
  test_idx = idx[int(0.85*n):]
  df['train'] = df['test'] = df['val'] = 0
  df.loc[train_idx, 'train'] = 1
  df.loc[test_idx, 'test'] = 1
  df.loc[val_idx, 'val'] = 1

In [ ]:
def train(training_data, critcol, iter):
  model.train()
  optimizer.zero_grad()
  print("[----------------------------------------------------------------------------------------------------]",
        end="\r[")
  pinterval = pmark = (len(iter))/101
  for idx in range(len(iter)):
    i = iter[idx]
    if idx > pmark:
      print("-", end='')
      pmark += pinterval

    data = from_smiles(training_data.get("smiles").iloc[i])
    data = data.to(device)
    data.x = data.x.type(torch.float)

    y = torch.Tensor(training_data[critcol].iloc[i])
    y_hat = model.forward(data.x, data.edge_index, edge_attr=data.edge_attr)
    y_pred = torch.sigmoid(y_hat)

    loss = model.criterion(y_pred, y)
    loss.backward()
  print("]", end ="\r")
  optimizer.step()

  global g_loss, epochs
  g_loss = loss
  epochs += 1

  return float(loss)

def test(testing_data, critcol):
  model.eval()
  test_cases = list(range(len(testing_data)))
  reals = []
  preds = []
  print("Test Eval:", end=" ")
  with torch.no_grad():
    for i in test_cases:
      data = from_smiles(testing_data.get("smiles").iloc[i])
      data = data.to(device)
      data.x = data.x.type(torch.float)

      y = torch.Tensor(testing_data[critcol].iloc[i])
      y_hat = model.forward(data.x, data.edge_index, edge_attr=data.edge_attr)
      y_pred = torch.sigmoid(y_hat)

      reals.append(y)
      preds.append(y_pred)
  reals = torch.stack(reals)
  preds = torch.stack(preds)
  auroc = model.auc_roc(preds, reals.int())
  acc = model.accuracy(preds, reals.int())
  f1 = model.f1_score(preds, reals.int())

  print("accuracy=" + str(round(float(acc), 5)) + ",",
        "f1=" + str(round(float(f1), 5)) + ",",
        "auc_roc=" + str(round(float(auroc), 5)),
        end=".")
  print("")

def val(val_data, critcol):
    loss = 0
    model.eval()
    with torch.no_grad():
        for i in range(len(val_data)):
            data = from_smiles(val_data.get("smiles").iloc[i])
            data = data.to(device)
            data.x = data.x.type(torch.float)

            y = torch.Tensor(val_data[critcol].iloc[i])
            y_hat = model.forward(data.x, data.edge_index, edge_attr=data.edge_attr)
            y_pred = torch.sigmoid(y_hat)

            loss += model.criterion(y_pred, y)
    return loss / len(val_data)

In [ ]:
# clintox training
trainset = df_clintox[df_clintox['train']==1]
valset = df_clintox[df_clintox['val']==1]
criterion = ['FDA_APPROVED', 'CT_TOX']
model = model_clintox
optimizer = optimizer_clintox

for i in range(100):
  iter = list(range(len(trainset)))
  shuffle(iter)
  print("Round", str(i+1), "gives us:", train(trainset, criterion, iter))
  test(valset, criterion)

In [ ]:
torch.save({'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': g_loss,},
               "/content/drive/Shareddrives/cs_89_network_science/saves/Bclintox")

In [ ]:
import pickle

In [ ]:
k = 5

In [ ]:
# Load the VGAE model
#model_vgae = VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=18, num_layers=2)) # 18 for Amodel; 64 for Bmodel q
model_vgae_clintox = VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=64, num_layers=2))
model_vgae_tox21 = VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=64, num_layers=2))

checkpoint = torch.load("/content/drive/Shareddrives/cs_89_network_science/saves/Bmodel") # eschew fine-tuned for now
model_vgae_clintox.load_state_dict(checkpoint['model_state_dict'])
model_vgae_tox21.load_state_dict(checkpoint['model_state_dict'])

with open("/content/drive/Shareddrives/cs_89_network_science/saves/128dim_pretrain_save", "rb") as f:
  lm = pickle.load(f)
model_vgae_clintox = lm['model']

#epochs = checkpoint['epoch']
#g_loss = checkpoint['loss']

# Initialize the classifiers
model_clintox = VGAEClassifier(model_vgae_clintox, 2)
optimizer_clintox = model_clintox.configure_optimizers()

model_tox21 = VGAEClassifier(model_vgae_tox21, 12)
optimizer_tox21 = model_tox21.configure_optimizers()


# Set to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Set to run on", device, end=".")
model_clintox = model_clintox.to(device)
model_tox21 = model_tox21.to(device)

Set to run on cpu.

In [ ]:
# clintox training
df_clintox_k = pd.read_csv("/content/drive/Shareddrives/cs_89_network_science/data/clintox_fivefoldsplit#" + str(k) + ".csv")
trainset = df_clintox_k[df_clintox_k['train']==1].reset_index(drop=True)
valset = df_clintox_k[df_clintox_k['val']==1].reset_index(drop=True)
testset = df_clintox_k[df_clintox_k['test']==1].reset_index(drop=True)
criterion = ['FDA_APPROVED', 'CT_TOX']
model = model_clintox
optimizer = optimizer_clintox

val_loss = 99999999
for i in range(100):
  iter = list(range(len(trainset)))
  shuffle(iter)
  print("Round", str(i+1), "gives us:", train(trainset, criterion, iter), end=". ")
  new_val_loss = val(valset, criterion)
  if new_val_loss < val_loss:
    print("New Checkpoint:", float(new_val_loss), end="; ")
    val_loss = new_val_loss
    torch.save({'epoch': i+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,},
               "/content/drive/Shareddrives/cs_89_network_science/saves/large_clintox")
  test(testset, criterion)

## Results
128-dim model with full epoch of pubchem.
### Clintox Results
Recorded at 100 epochs. "large_clintox"

k=1; test loss=0.5417; test acc=0.9396; test f1=0.4698; test auroc=0.3245;

k=2; test loss=0.5498; test acc=0.94631; test f1=0.48797; test auroc=0.20531;

k=3; test loss=0.5433; test acc=0.90268; test f1=0.47535; test auroc=0.3946;

k=4; test loss=0.5274; test acc=0.92617; test f1=0.48264; test auroc=0.33376;

k=5; test loss=; test acc=; test f1=; test auroc=;


## Results
Note these are of the 64-dim model, with very limited pretraining (exact epoch number tbc).
### Clintox Results
Recorded at 100 epochs. "C_clintox"

k=1; test loss=0.6249; test acc=0.93289; test f1=0.46309; test auroc=;
(acc & f1 held at epochs 56-68, 69-88, 89-92, 93-100; loss continued decreasing)

(Out of curiosity, recorder at 200 epochs):
test loss=0.5748; test acc=0.93624; test f1=0.46644; test auroc=;
(acc & f1 held at 101-126, 127-200)

k=3; test loss=; test acc=; test f1=; test auroc=;

k=4; test loss=; test acc=; test f1=; test auroc=;

k=5; test loss=; test acc=; test f1=; test auroc=;

In [ ]:
# Flatten the results and create a DataFrame
results = {'clintox_1': [0.5417, 0.9396, 0.4698, 0.3245, 100],
           'clintox_2': [0.5498, 0.94631, 0.48797, 0.20531, 100],
           'clintox_3': [0.5433, 0.90268, 0.47535, 0.3946, 100],
           'clintox_4': [0.5274, 0.92617, 0.48264, 0.33376, 100],
           'clintox_5': [0.5311, 0.91275, 0.48084, 0.42356, 100]
           }
df_results = pd.DataFrame.from_dict(results, orient='index', columns=['test_loss', 'test_acc', 'test_f1', 'test_auc_roc', 'epochs'])

# Display the resulting DataFrame
print(df_results)
df_results.to_csv('/content/drive/Shareddrives/cs_89_network_science/results/VGAE_GAT.csv')

In [ ]:
checkpoint = torch.load("/content/drive/Shareddrives/cs_89_network_science/saves/C_clintox") # eschew fine-tuned for now
model = VGAEClassifier(VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=64, num_layers=2)), 2)
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
test(testset, criterion)

In [ ]:
# tox21 training
df_tox21_nt0 = df_tox21.fillna(0)
trainset = df_tox21_nt0[df_tox21_nt0['train']==1]
valset = df_tox21_nt0[df_tox21_nt0['val']==1]
criterion = ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD','NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5',
             'SR-HSE', 'SR-MMP', 'SR-p53']
model = model_tox21
optimizer = optimizer_tox21

for i in range(50):
  iter = list(range(len(trainset)))
  shuffle(iter)
  print("Round", str(i+1), "gives us:", train(trainset, criterion, iter))
  test(valset, criterion)

In [ ]:
torch.save({'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': g_loss,},
               "/content/drive/Shareddrives/cs_89_network_science/saves/tox21_NA0")

## Results
128-dim model with full epoch of pubchem.
### Tox21 Results
Recorded at 100 epochs. "large_clintox"
72. New Checkpoint: 0.7655649781227112; Test Eval: accuracy=0.94184, f1=0.0, auc_roc=0.36136,.
k=1; test loss=0.5417; test acc=0.9396; test f1=0.4698; test auroc=0.3245;

In [ ]:
k = 5

# Load the VGAE model
#model_vgae = VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=18, num_layers=2)) # 18 for Amodel; 64 for Bmodel q
model_vgae_tox21 = VGAE(encoder=GATEncoder(in_channels=9, hidden_dim=64, num_layers=2))

checkpoint = torch.load("/content/drive/Shareddrives/cs_89_network_science/saves/Bmodel") # eschew fine-tuned for now
model_vgae_tox21.load_state_dict(checkpoint['model_state_dict'])

with open("/content/drive/Shareddrives/cs_89_network_science/saves/128dim_pretrain_save", "rb") as f:
  lm = pickle.load(f)
model_vgae_tox21 = lm['model']

# Initialize the classifiers
model_tox21 = VGAEClassifier(model_vgae_tox21, 12)
optimizer_tox21 = model_tox21.configure_optimizers()

# Set to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Set to run on", device, end=".")
model_tox21 = model_tox21.to(device)

# clintox training
df_k = pd.read_csv("/content/drive/Shareddrives/cs_89_network_science/data/tox21_fivefoldsplit#" + str(k) + ".csv").fillna(0)
trainset = df_k[df_k['train']==1].reset_index(drop=True)
valset = df_k[df_k['val']==1].reset_index(drop=True)
testset = df_k[df_k['test']==1].reset_index(drop=True)
criterion = ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD','NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']
model = model_tox21
optimizer = optimizer_tox21

val_loss = 99999999
for i in range(50):
  iter = list(range(len(trainset)))
  shuffle(iter)
  print("Round", str(i+1), "gives us:", train(trainset, criterion, iter), end=". ")
  new_val_loss = val(valset, criterion)
  if new_val_loss < val_loss:
    print("New Checkpoint:", float(new_val_loss), end="; ")
    val_loss = new_val_loss
    torch.save({'epoch': i+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,},
               "/content/drive/Shareddrives/cs_89_network_science/saves/large_clintox")
  test(testset, criterion)

In [ ]:
test(trainset, criterion)

Test Eval: accuracy=0.93837, f1=0.0, auc_roc=0.37312.


In [ ]:
# Flatten the results and create a DataFrame
results = {'clintox_1': [0.5417, 0.9396, 0.4698, 0.3245, 100],
           'clintox_2': [0.5498, 0.94631, 0.48797, 0.20531, 100],
           'clintox_3': [0.5433, 0.90268, 0.47535, 0.3946, 100],
           'clintox_4': [0.5274, 0.92617, 0.48264, 0.33376, 100],
           'clintox_5': [0.5311, 0.91275, 0.48084, 0.42356, 100],
           'tox21_1': [0.7757, 0.94184, 0.0, 0.36223, 50],
           'tox21_2': [0.7668, 0.93654, 0.0, 0.36198, 50],
           'tox21_3': [0.7668, 0.93071, 0.0, 0.34779, 50],
           'tox21_4': [0.7747, 0.93914, 0.0, 0.38622, 50],
           'tox21_5': [0.7811, 0.94085, 0.0, 0.3702]
           }
df_results = pd.DataFrame.from_dict(results, orient='index', columns=['test_loss', 'test_acc', 'test_f1', 'test_auc_roc', 'epochs'])

# Display the resulting DataFrame
print(df_results)
df_results.to_csv('/content/drive/Shareddrives/cs_89_network_science/results/VGAE_GAT.csv')

In [ ]:
with open("temp2", "wb") as f:
  pickle.dump(model, f)

In [ ]:
(0.36223 + 0.36198 + 0.34779 + 0.38622 + 00.37020)/5